<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget

In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

In [5]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [6]:
spacex_df['class'].to_numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0])

Now, you can take a look at what are the coordinates for each site.
## Note: Had to add in class to launch_sites_df

In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class','Landing Outcome']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long','class']]
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a red circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a red circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


## Original lab did **not** create a popup label correctly.

In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

launch_sites_df
for row in launch_sites_df.index:
    name = launch_sites_df.iloc[row]['Launch Site']
    lat = launch_sites_df.iloc[row]['Lat']
    long = launch_sites_df.iloc[row]['Long']
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name

    circle = folium.Circle([lat, long],
                           radius=100,
                           color='#d35400',
                           fill=True).add_child(folium.Popup(name))
    
    marker = folium.map.Marker([lat, long],
                               icon=DivIcon(icon_size=(20,20),
                                            icon_anchor=(0,0),
                                            html=html)
                              )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


## Launch site VAFB is farther north than the others at $34.6^{\circ} N$. All of the others are the about the same distance from the equator near $28.6^{\circ} N$.

## All launch sites are very close to a coast.

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [12]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,Landing Outcome
46,KSC LC-39A,28.573255,-80.646895,1,Success (drone ship)
47,KSC LC-39A,28.573255,-80.646895,1,Success (drone ship)
48,KSC LC-39A,28.573255,-80.646895,1,Success (drone ship)
49,CCAFS SLC-40,28.563197,-80.576820,1,Success (ground pad)
50,CCAFS SLC-40,28.563197,-80.576820,1,Success (ground pad)
51,CCAFS SLC-40,28.563197,-80.576820,0,Controlled (ocean)
52,CCAFS SLC-40,28.563197,-80.576820,0,No attempt
53,CCAFS SLC-40,28.563197,-80.576820,0,No attempt
54,CCAFS SLC-40,28.563197,-80.576820,1,Success (drone ship)
55,CCAFS SLC-40,28.563197,-80.576820,0,No attempt


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [13]:
marker_cluster = MarkerCluster()

In [14]:
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value
## Create a new column in **`spacex_df`** dataframe.
## Changed the marker colors to green (success) and yellow (fail) as I am colorblind and couldn't see the difference.

In [15]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Function to assign color to launch outcome

In [16]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'yellow'
    
spacex_df['marker_color'] = spacex_df.apply(lambda x: assign_marker_color(x['class']), axis=1)
spacex_df['marker_color'] = spacex_df['marker_color']
spacex_df.tail(10)

,Launch Site,Lat,Long,class,Landing Outcome,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,Success (drone ship),green
47,KSC LC-39A,28.573255,-80.646895,1,Success (drone ship),green
48,KSC LC-39A,28.573255,-80.646895,1,Success (drone ship),green
49,CCAFS SLC-40,28.563197,-80.576820,1,Success (ground pad),green
50,CCAFS SLC-40,28.563197,-80.576820,1,Success (ground pad),green
51,CCAFS SLC-40,28.563197,-80.576820,0,Controlled (ocean),yellow
52,CCAFS SLC-40,28.563197,-80.576820,0,No attempt,yellow
53,CCAFS SLC-40,28.563197,-80.576820,0,No attempt,yellow
54,CCAFS SLC-40,28.563197,-80.576820,1,Success (drone ship),green
55,CCAFS SLC-40,28.563197,-80.576820,0,No attempt,yellow


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [17]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
marker_cluster = MarkerCluster()
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Recreates the launch site circles and markers.
launch_sites_df
for row in launch_sites_df.index:
    name = launch_sites_df.iloc[row]['Launch Site']
    lat = launch_sites_df.iloc[row]['Lat']
    long = launch_sites_df.iloc[row]['Long']
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name

    circle = folium.Circle([lat, long],
                           radius=50,
                           color='#d35400',
                           fill=True).add_child(folium.Popup(name))
    
    marker = folium.map.Marker([lat, long],
                               icon=DivIcon(icon_size=(20,20),
                                            icon_anchor=(0,0),
                                            html=html)
                              )
    site_map.add_child(circle)
    site_map.add_child(marker)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.map.Marker([record['Lat'], record['Long']],
                               # Create an icon as a text label
                               icon=folium.Icon(color='white', icon_color=record['marker_color']))

    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [20]:
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

coords = {'CCAFS LC-40':{'coast':[28.56232,-80.56779],
                         'highway':[28.56277,-80.57064],
                         'rail':[28.57205,-80.58528],
                         'city':[28.39804,-80.60751]},
          'CCAFS SLC-40':{'coast':[28.56232,-80.56779],
                          'highway':[28.56277,-80.57064],
                          'rail':[28.57205,-80.58528],
                          'city':[28.39804,-80.60751]},
          'KSC LC-39A':{'coast':[28.57641,-80.57126],
                        'highway':[28.5735,-80.65403],
                        'rail':[28.57314,-80.65579],
                        'city':[28.61406,-80.80934]},
          'VAFB SLC-4E':{'coast':[34.63716,-120.62491],
                         'highway':[34.70468,-120.56944],
                         'rail':[34.63603,-120.62405],
                         'city':[34.64422,-120.45798]}}


launch_sites_coords = {'CCAFS LC-40':[28.562302,-80.577356],
                       'CCAFS SLC-40':[28.563197,-80.576820],
                       'KSC LC-39A':[28.573255,-80.646895],
                       'VAFB SLC-4E':[34.632834,-120.610745]}

In [22]:
distance_coastline = {}
distance_highway = {}
distance_rail = {}
distance_city = {}

for site in launch_sites_coords.keys():
    distance_coastline[site] = calculate_distance(launch_sites_coords[site][0],
                                                  launch_sites_coords[site][1],
                                                  coords[site]['coast'][0],
                                                  coords[site]['coast'][1])
    distance_highway[site] = calculate_distance(launch_sites_coords[site][0],
                                                  launch_sites_coords[site][1],
                                                  coords[site]['highway'][0],
                                                  coords[site]['highway'][1])
    distance_rail[site] = calculate_distance(launch_sites_coords[site][0],
                                                  launch_sites_coords[site][1],
                                                  coords[site]['rail'][0],
                                                  coords[site]['rail'][1])
    distance_city[site] = calculate_distance(launch_sites_coords[site][0],
                                                  launch_sites_coords[site][1],
                                                  coords[site]['city'][0],
                                                  coords[site]['city'][1])

print(f'Coast: {distance_coastline}')
print(f'Highway: {distance_highway}')
print(f'Rail: {distance_rail}')
print(f'City: {distance_city}')

Coast: {'CCAFS LC-40': 0.9345324554154533, 'CCAFS SLC-40': 0.8875401794682829, 'KSC LC-39A': 7.39645314121483, 'VAFB SLC-4E': 1.3827837004687153}
Highway: {'CCAFS LC-40': 0.6581659493197421, 'CCAFS SLC-40': 0.6056035857836531, 'KSC LC-39A': 0.6974981209287902, 'VAFB SLC-4E': 8.83973412557668}
Rail: {'CCAFS LC-40': 1.3322320148781461, 'CCAFS SLC-40': 1.2855630845538104, 'KSC LC-39A': 0.8689826268926583, 'VAFB SLC-4E': 1.268495717215634}
City: {'CCAFS LC-40': 18.507156791944542, 'CCAFS SLC-40': 18.613816633219695, 'KSC LC-39A': 16.501457676202133, 'VAFB SLC-4E': 14.037508321496414}


*TODO*
- [x] After obtained its coordinate, create a `folium.Marker` to show the distance \
- [x] Draw a `PolyLine` between a launch site to the selected coastline point \
- [x] Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first

In [23]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)

def create_poly(launch_coords, object_coords, distance_type, distances, site_map):
    for site in ['CCAFS LC-40','KSC LC-39A','VAFB SLC-4E']:
        coord_origin = [launch_coords[site][0], launch_coords[site][1]]
        coord_destination = [coords[site][distance_type][0], coords[site][distance_type][1]]
        distance = distances[site]
        
        distance_marker = folium.Marker(coord_destination,
                                        icon=DivIcon(icon_size=(20,20),
                                                     icon_anchor=(0,0),
                                                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
                                                     )
                                       )
        site_map.add_child(distance_marker)
        
        locations = [coord_origin, coord_destination]
        
        folium.PolyLine(locations=locations, weight=1,color='blue').add_to(site_map)

In [24]:
create_poly(launch_sites_coords, coords, 'coast', distance_coastline, site_map)
create_poly(launch_sites_coords, coords, 'highway', distance_highway, site_map)
create_poly(launch_sites_coords, coords, 'rail', distance_rail, site_map)
create_poly(launch_sites_coords, coords, 'city', distance_city, site_map)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.

# Launch sites are close proximity to coasts. This is to ensure that the launches happen over the ocean such that the rockets or debris fall into the ocean.

In [25]:
print(f'Coast: {distance_coastline}')

Coast: {'CCAFS LC-40': 0.9345324554154533, 'CCAFS SLC-40': 0.8875401794682829, 'KSC LC-39A': 7.39645314121483, 'VAFB SLC-4E': 1.3827837004687153}


# Launch sites are also in close proximity to railways and/or highways. This is so it is easy to transport the rockets after they are built to the launch pads.

In [26]:
print(f'Highway: {distance_highway}')
print(f'Rail: {distance_rail}')

Highway: {'CCAFS LC-40': 0.6581659493197421, 'CCAFS SLC-40': 0.6056035857836531, 'KSC LC-39A': 0.6974981209287902, 'VAFB SLC-4E': 8.83973412557668}
Rail: {'CCAFS LC-40': 1.3322320148781461, 'CCAFS SLC-40': 1.2855630845538104, 'KSC LC-39A': 0.8689826268926583, 'VAFB SLC-4E': 1.268495717215634}


# Launch sites are $>= 10$ miles from cities. This is to ensure that if something happens on launch, the rockets or debris have a very small chance of falling onto the city.

In [27]:
print(f'City: {distance_city}')

City: {'CCAFS LC-40': 18.507156791944542, 'CCAFS SLC-40': 18.613816633219695, 'KSC LC-39A': 16.501457676202133, 'VAFB SLC-4E': 14.037508321496414}


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
